# Single objective optimization with (sequential) linear programming and `PESTPP-OPT`: bringing in risk evaluation 

In [ ]:
import os
import sys
sys.path.insert(0,"..")
import numpy as np
import matplotlib.pyplot as plt
import pyemu
print(pyemu.__file__)
import flopy
import platform
from pathlib import Path
import shutil
import pandas as pd
from IPython import display


In [ ]:
display.Image("./mv_schematic.png",width=650) 

## Problem setup
 - two city wells in the south of the domain, in combination need to provide 150,000 ft^3/d of water for a city but would like as much as possible
 - the northern well would like to produce 50,000 ft^3/d for a fancy brewery making nettle-mead syrup and moss beer #soHipster
 - The two stream gages can experience some depletion, but only up to 30%

# Let's build on the previous optimization and make use of the ensemble (stack) from running iES

In [ ]:
pstroot = 'mv_opt_risk.05'

In [ ]:
thisdir = os.getcwd()

# Let's start with our OPT setup - we need to make some changes to it

In [ ]:
template_ws = Path('./simple_opt')
new_ws = Path('./simple_opt_risk')

In [ ]:
if os.path.exists(new_ws):
    shutil.rmtree(new_ws)
shutil.copytree(template_ws, new_ws)

In [ ]:
pst = pyemu.Pst(str(new_ws / 'mv_opt.pst'))

# To consider uncertainty of the objective function outputs, we need to bring in the stack using a couple options

### first let's copy over the ensemble files from the iES run directory

In [ ]:
[shutil.copy2(f'./master_ies_simple/at.5.{cf}.csv',new_ws/ f'at.3.{cf}.stack.csv') for cf in ['obs','par']]


In [ ]:
pst.pestpp_options['opt_recalc_chance_every'] = 100 # let's assume the stack is ok and doesn't need recalculating
pst.pestpp_options['opt_par_stack'] = 'at.3.par.stack.csv'
pst.pestpp_options['opt_obs_stack'] = 'at.3.obs.stack.csv'
pst.pestpp_options['opt_stack_size'] = 100
pst.pestpp_options['opt_risk'] = 0.05


## Also have to free up parameters

In [ ]:
pars = pst.parameter_data
pars.partrans='none'

In [ ]:
obs=pst.observation_data
obs

In [ ]:
pst.control_data.noptmax = 1
pst.write(str(new_ws / f'{pstroot}.pst'), version=2)

In [ ]:
pyemu.os_utils.run(f'pestpp-opt {pstroot}.pst',cwd=new_ws)

## let's sweep across risk and see what happens


In [ ]:
pst.pestpp_options['hotstart_resfile'] = 'mv_opt_risk.05.1.jcb.rei'
pst.pestpp_options['base_jacobian'] = 'mv_opt_risk.05.1.jcb'
pst.pestpp_options['opt_skip_final'] = True
pst.control_data.noptmax = 1


In [ ]:
os.chdir(new_ws)
riskroots = []
risks = np.linspace(0.01,.99,99)
for i in risks:
    pst.pestpp_options['opt_risk'] = i
    pstroot = f'mv_sweep_{i:0.2f}'
    print(pstroot)
    riskroots.append(pstroot)
    pst.write( f'{pstroot}.pst', version=2)
    pyemu.os_utils.run(f'pestpp-opt {pstroot}.pst')

os.chdir(thisdir)

In [ ]:
os.chdir(thisdir)

In [ ]:
risks

In [ ]:
feas = []
for cf in riskroots:
    if 'infeasi' in  ''.join([i.strip().lower() for i in open(new_ws/ f'{cf}.rec', 'r').readlines()]):
        feas.append(False)
    else:
        feas.append(True)

In [ ]:
feas

In [ ]:
stream_ds_mod_shift = []
stream_ds_meas = []
stream_ds_mod = []

for root in riskroots:
    print(root, end='\r')
    if (new_ws / f'{root}.1.est+chance.rei').exists():
        stream_ds_meas.append(float(open(new_ws / f'{root}.1.est+chance.rei').readlines()[4].strip().split()[2]))
        stream_ds_mod_shift.append(float(open(new_ws / f'{root}.1.est+chance.rei').readlines()[4].strip().split()[3]))
        stream_ds_mod.append(float(open(new_ws / f'{root}.1.est.rei').readlines()[4].strip().split()[3]))
    else:
        stream_ds_meas.append(stream_ds_meas[0])
        stream_ds_mod.append(np.nan)
        stream_ds_mod_shift.append(np.nan)

stream_pf_mod_shift = []
stream_pf_meas = []
stream_pf_mod = []

for root in riskroots:
    print(root, end='\r')
    if (new_ws / f'{root}.1.est+chance.rei').exists():
        stream_pf_meas.append(float(open(new_ws / f'{root}.1.est+chance.rei').readlines()[5].strip().split()[2]))
        stream_pf_mod_shift.append(float(open(new_ws / f'{root}.1.est+chance.rei').readlines()[5].strip().split()[3]))
        stream_pf_mod.append(float(open(new_ws / f'{root}.1.est.rei').readlines()[5].strip().split()[3]))
        
    else:
        stream_pf_meas.append(stream_pf_meas[0])
        stream_pf_mod.append(np.nan)
        stream_pf_mod_shift.append(np.nan)
        

In [ ]:
risk_shift_results = pd.DataFrame(
    index=risks,
    data = {
        'feas':feas,
        'ds_mod': stream_ds_mod,
        'ds_mod_risk': stream_ds_mod_shift,
        'ds_meas': stream_ds_meas,
        'pf_mod': stream_pf_mod,
        'pf_mod_risk': stream_pf_mod_shift,
        'pf_meas': stream_pf_meas,
        
    }
)

In [ ]:
risk_shift_results

In [ ]:
ax = risk_shift_results[['ds_meas','ds_mod','ds_mod_risk']].plot()
plt.xlabel('risk')
ax.fill_between(risk_shift_results.index,
                ax.get_ylim()[0],
                ax.get_ylim()[1],
                where=(~risk_shift_results.feas), color='red', alpha=0.5)


In [ ]:
ax = risk_shift_results[['pf_meas','pf_mod','pf_mod_risk']].plot()
plt.xlabel('risk')
ax.fill_between(risk_shift_results.index,
                ax.get_ylim()[0],
                ax.get_ylim()[1],
                where=(~risk_shift_results.feas), color='red', alpha=0.5)

### What about the decision variables? How are they changing across risk profiles?

In [ ]:
dvs = []

for root in riskroots:
    print(root, end='\r')
    if (new_ws / f'{root}.1.par').exists():
        cp = pd.read_csv(new_ws / f'{root}.1.par',  sep=r'\s+', header=None, skiprows=1,
                        names=['parname','parval'], usecols=[0,1])
        dvs.append(cp.loc[cp.parname.str.contains('wflux')].T.to_numpy()[1])

    else:
        dvs.append(np.ones(3)*np.nan)

dv_df = pd.DataFrame(
    index=risks,
    data = np.array(dvs).astype(float),
    columns=cp.loc[cp.parname.str.contains('wflux')].T.to_numpy()[0]
)

In [ ]:
dv_df['feas'] = feas
dv_df

In [ ]:
ax = dv_df[['wflux_k:4_i:32_j:5','wflux_k:4_i:5_j:14','wflux_k:4_i:34_j:15']].plot()
plt.xlabel('risk')
plt.ylabel('Pumping')
ax.fill_between(risk_shift_results.index,
                ax.get_ylim()[0],
                ax.get_ylim()[1],
                where=(~dv_df.feas), color='red', alpha=0.5)